In [1]:
import arcpy
import arcpy, os, sys
import pandas as pd

In [2]:
output_folder = r"your path\Devide_Polygons.gdb"
output_polygon = "split_polygons" 
spatial_ref = arcpy.SpatialReference(2284) 

In [3]:
# Create a new polygon feature class to store the cut polygons

if arcpy.Exists(output_polygon):
    arcpy.Delete_management(output_polygon)
arcpy.management.CreateFeatureclass( output_folder,output_polygon, "POLYGON", spatial_reference=spatial_ref)


<Result 'C:\\Users\\bsb5pw\\OneDrive - University of Virginia\\Desktop\\github\\Divide_Room_Polygons_Based_On_the_Area_Percentages\\Devide_Polygons\\Devide_Polygons.gdb\\split_polygons'>

In [4]:

arcpy.AddField_management(
    in_table=output_polygon,  # The table or feature class to which you're adding the field
    field_name="Employee_ID",   # The name of the new field
    field_type="TEXT",       # Field type (e.g., TEXT, FLOAT, DOUBLE, SHORT, LONG, DATE, etc.)
    field_length=50          # Optional: set the length if it's a TEXT field
)
arcpy.AddField_management(
    in_table=output_polygon,  # The table or feature class to which you're adding the field
    field_name="Building_ID",   # The name of the new field
    field_type="TEXT",       # Field type (e.g., TEXT, FLOAT, DOUBLE, SHORT, LONG, DATE, etc.)
    field_length=50          # Optional: set the length if it's a TEXT field
)
arcpy.AddField_management(
    in_table=output_polygon,  # The table or feature class to which you're adding the field
    field_name="Floor_ID",   # The name of the new field
    field_type="TEXT",       # Field type (e.g., TEXT, FLOAT, DOUBLE, SHORT, LONG, DATE, etc.)
    field_length=50          # Optional: set the length if it's a TEXT field
)
arcpy.AddField_management(
    in_table=output_polygon,  # The table or feature class to which you're adding the field
    field_name="Room_ID",   # The name of the new field
    field_type="TEXT",       # Field type (e.g., TEXT, FLOAT, DOUBLE, SHORT, LONG, DATE, etc.)
    field_length=50          # Optional: set the length if it's a TEXT field
)

arcpy.AddField_management(
    in_table=output_polygon,  # The table or feature class to which you're adding the field
    field_name="PropertyLocation",   # The name of the new field
    field_type="TEXT",       # Field type (e.g., TEXT, FLOAT, DOUBLE, SHORT, LONG, DATE, etc.)
    field_length=50          # Optional: set the length if it's a TEXT field
)
# Add a single field
arcpy.AddField_management(
    in_table=output_polygon,  # The table or feature class to which you're adding the field
    field_name="percentage",   # The name of the new field
    field_type="Double",       # Field type (e.g., TEXT, FLOAT, DOUBLE, SHORT, LONG, DATE, etc.)
    field_length=50          # Optional: set the length if it's a TEXT field
)



<Result 'split_polygons'>

In [ ]:
pd.options.display.max_columns = None


def feature_class_to_dataframe(input_fc: str, input_fields: list = None, query: str = ""):
    """Converts a feature class or a table to a pandas dataframe. If 
    no input fields are specified, all fields
    will be included. If a query is specified, only those
    features will be included in the dataframe.

    This is an excellent function to use when exploring data
    without having to queue up ArcGIS Pro. Particularly good
    for using pandas to generate unique field values.

    Args:
        input_fc (string): path to the input feature class
        input_fields (list, optional): List of fields for dataframe. 
            Defaults to None.
        query (str, optional): Pandas query. Defaults to "".

    Returns:
        Pandas Dataframe: Dataframe of feature class
    """

    from arcpy import Describe, ListFields
    from arcpy.da import SearchCursor
    from pandas import DataFrame

    # get list of fields if desired fields specified
    OIDFieldName = Describe(input_fc).OIDFieldName
    if input_fields:
        final_fields = [OIDFieldName] + input_fields

    # use all fields if no fields specified
    else:
        final_fields = [field.name for field in ListFields(input_fc)]

    # build dataframe row by row using search cursor
    data = [row for row in SearchCursor(
        input_fc, final_fields, where_clause=query)]
    fc_dataframe = DataFrame(data, columns=final_fields)

    # set index to object id
    fc_dataframe = fc_dataframe.set_index(OIDFieldName, drop=True)
    
    return fc_dataframe


In [6]:
# shared room percentage saved in the table(Shared_Room_Table)
df_Pls_to_rooms = feature_class_to_dataframe("Shared_Room_Table")

In [7]:
df_Pls_to_rooms

,Employee_ID,PropertyLocation,Building_Code,Room_ID,floor,Room_Percent
OBJECTID,,,,,,
1,CAC4AN,11571007,1157,1007,01,50
2,LAP5B,11571007,1157,1007,01,30
3,KPJ8AK,11571007,1157,1007,01,20
4,CAC4AN,11571007A,1157,1007A,01,60
5,KMN4MJ,11571110,1157,1110,01,80
6,BPH6N,11571110,1157,1110,01,20
7,KMN4MJ,11571127,1157,1127,01,50
8,SD8US,11571127,1157,1127,01,50
9,SD8US,11571128,1157,1128,01,90


In [9]:
# room polygons feature saved in the table(Shared_Room_Polygons)
# loop each room polygon and cut it by percentages which are saved in the table(Shared_Room_Table)
# the create the split polygons saved in the feature class(split_polygons)
# If the cut line passes through the vertices, the cut function will not work, and an error will occur. We use a try-except clause to handle this issue
with arcpy.da.SearchCursor('Shared_Room_Polygons', ["SHAPE@","PropertyLocation","Building_ID","Room_ID","floor"]) as pcursor:
   
   icursor_split_polygons = arcpy.da.InsertCursor("split_polygons",["SHAPE@","Employee_ID","Building_ID",
                                                                    "Floor_ID","Room_ID","PropertyLocation","percentage"])
   for prow in pcursor:
      polygon = prow[0]
      PropertyLocation=prow[1]
      percentages = df_Pls_to_rooms[df_Pls_to_rooms['PropertyLocation']==PropertyLocation][['Room_Percent']].squeeze().tolist()

      Building_ID=prow[2]
      Room_ID=prow[3]
      Floor_ID=prow[4]
      
      e = polygon.extent

      stepsize = (e.XMax-e.XMin)/1000
      leftXstart = e.XMin
      leftX = e.XMin + stepsize
      ymax = e.YMax
      ymin = e.YMin
      cutpoly = polygon
      df_one_room = df_Pls_to_rooms[df_Pls_to_rooms['PropertyLocation']==PropertyLocation]
      
      j =0
      for index,row in df_one_room.iterrows():
            Employee_ID= row[0]
            Room_Percent= row[5]
            print(Room_Percent)
            j=j+1
            if j<= len(percentages[:len(percentages)-1]):
                
                tol = 0
                while tol < Room_Percent:

                      point_1 = arcpy.Point(leftX, ymax)
                      point_2 = arcpy.Point(leftX, ymin)
                      points = [point_1,point_2]
                      pntarray = arcpy.Array(points)
                      pline = arcpy.Polyline(pntarray,arcpy.SpatialReference(2284))

                      if cutpoly.overlaps(pline) or cutpoly.crosses(pline):
                          try:
                                    cutlist = cutpoly.cut(pline)

                          except Exception as e:
                                    print("exception:")

                                    print("An error occurred:", e)
                                    pass 

                          tol = 100 * cutlist[1].area / polygon.area

                      leftX += stepsize


                cutpoly = cutlist[0]

                icursor_split_polygons.insertRow([cutlist[1],Employee_ID,Building_ID,Floor_ID,Room_ID,PropertyLocation,Room_Percent])


        # part 0 is on the right and part 1 is on the left of the split
      icursor_split_polygons.insertRow([cutlist[0],Employee_ID,Building_ID,Floor_ID,Room_ID,PropertyLocation,Room_Percent])
   del icursor_split_polygons
            
del pcursor

25
exception:
An error occurred: A polygon cut operation could not classify all parts of the polygon as left or right of the cutting line.
25
25
25
50
exception:
An error occurred: A polygon cut operation could not classify all parts of the polygon as left or right of the cutting line.
50
90
exception:
An error occurred: A polygon cut operation could not classify all parts of the polygon as left or right of the cutting line.
10
80
exception:
An error occurred: A polygon cut operation could not classify all parts of the polygon as left or right of the cutting line.
20
50
30
20
60
exception:
An error occurred: A polygon cut operation could not classify all parts of the polygon as left or right of the cutting line.
40
50
50
70
30
